# Different ways to make a query on mongoDB with ***C#***
The idea is create the same query using different techniques and try understanding the benefits of each one. 


## Setup 
___
***Reference***
* [mongo driver doc](https://docs.mongodb.com/drivers/csharp) 

### Dependencies

#### Mongo DB - DOCKER

In [1]:
#!pwsh
docker run -d -p 27017-27017:27017-27017 mongo 

#### Nuget packages

In [1]:
#r "nuget:MongoDB.Driver"

#### Client 

In [1]:

using MongoDB.Driver;

var client = new MongoClient("mongodb://localhost:27017");
var database = client.GetDatabase("firstDatabase");

### Creating Models

In [1]:
using MongoDB.Bson.Serialization.Attributes;

[BsonIgnoreExtraElements]
public class Collection{
    public string FirstProperty { get; set; }
    public int SecondProperty { get; set; }

}
[BsonIgnoreExtraElements]
public class FirstCollection : Collection{
    public IEnumerable<SecondCollection> SecondCollections { get; set; }
}

[BsonIgnoreExtraElements]
public class SecondCollection : Collection {
    public string Name { get; set; }
}

### Instanciate Collections

In [1]:
var firstCollection = database.GetCollection<FirstCollection>("firstCollection");
var secondCollection = database.GetCollection<SecondCollection>("secondCollection");


### Seeding

#### first Collection

In [1]:

var firstCollections = new List<FirstCollection>{
    new FirstCollection {FirstProperty = "First", SecondProperty = 1 },
    new FirstCollection {FirstProperty = "Second", SecondProperty = 2 },
    new FirstCollection {FirstProperty = "Third", SecondProperty = 3 },
    new FirstCollection {FirstProperty = "Fourth", SecondProperty = 4 }
};

firstCollection.InsertMany(firstCollections);

#### second Collection

In [1]:

var secondCollections = new List<SecondCollection>{
    new SecondCollection {FirstProperty = "First", SecondProperty = 1, Name = "I" },
    new SecondCollection {FirstProperty = "Second", SecondProperty = 2 , Name = "II"},
    new SecondCollection {FirstProperty = "Third", SecondProperty = 3 , Name = "III"},
    new SecondCollection {FirstProperty = "Fourth", SecondProperty = 4 , Name = "IV"}
};

secondCollection.InsertMany(secondCollections);

### Utils methods

In [1]:

    public void PrintResult<TResult>(IEnumerable<TResult> results, Func<TResult,string> output){
        Console.WriteLine($"Result Count: {results.Count()}");
        results.ToList().ForEach(res => Console.WriteLine(output(res)));
    }


## Queries:

1. Simple query: Filter ***FirstCollection*** by property ***'FirstProperty'*** not equals ***'First'*** and the property ***'SecondProperty'*** in 1,2 and 4.
2. Aggragate query: Aggragate the  ***FirstCollection*** with ***SecondCollection*** by ***'FirstProperty'*** and filter ***FirstCollection*** by property ***'FirstProperty'*** not equals ***'First'***, selecting only ***SecondCollection's*** Name property.

### Using ***Filters*** Builder 
***Reference:***
* [Mongo DB Blog](https://www.mongodb.com/blog/post/quick-start-c-and-mongodb--read-operations)

#### Simple Query 


In [1]:
var numbers = new List<int>{1,4,2};

var collectionInFilter = Builders<FirstCollection>.Filter.In(collect => collect.SecondProperty,numbers);
var collectionEqFilter = Builders<FirstCollection>.Filter.Eq(collect => collect.FirstProperty,"First");
var collectionNotInFilter = Builders<FirstCollection>.Filter.Not(collectionEqFilter);
var collectionAndFilter = Builders<FirstCollection>.Filter.And(collectionInFilter,collectionNotInFilter);

var findFilterResult = firstCollection.FindSync(collectionAndFilter).ToList();

PrintResult(findFilterResult, result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondProperty}");

#### Agreggate Query 

In [1]:
var numbers = new List<int>{1,4,2};

var collectionEqFilter = Builders<FirstCollection>.Filter.Eq(collect => collect.FirstProperty,"First");
var collectionNotInFilter = Builders<FirstCollection>.Filter.Not(collectionEqFilter);


var findFilterResult = firstCollection.Aggregate()
    .Match(collectionNotInFilter)
    .Lookup(
        foreignCollection: secondCollection,
        first => first.SecondProperty,
        second => second.SecondProperty,
        @as : (FirstCollection first) => first.SecondCollections)
    .Project(x => new {Names = x.SecondCollections.Select(second => second.Name)})
    .ToList()
    .SelectMany(project => project.Names);

PrintResult(findFilterResult, result => $"Name: {result}");

### Using fluent ***Mongo driver*** Linq 
___
***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/)

#### Simple Query 


In [1]:
var numbers = new List<int>{1,4,2};

var fluentResult = firstCollection.AsQueryable()
    .Where(collect => 
        !collect.FirstProperty.Equals("First")
        && numbers.Contains(collect.SecondProperty))
    .ToList();

PrintResult(fluentResult, result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondProperty}");


#### Agreggate Query 

In [1]:
var numbers = new List<int>{1,4,2};

var fluentResult = firstCollection.AsQueryable()
    .Where(collect => !collect.FirstProperty.Equals("First"))
    .Join(secondCollection.AsQueryable(),
        first => first.SecondProperty,
        second => second.SecondProperty,
        (_,second) => second)
    .Select(second => second.Name)
    .ToList();

PrintResult(fluentResult, result => $"Name: {result}");


### Using DSL ***Mongo driver*** Linq 
***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/) 
* [LINQ as DSL](https://app.getpocket.com/read/652237004)
___

#### Simple Query 
Create a simple filter with ***Linq*** as ***DSL*** not a good choice, because loses the great deal of this technique, the ***readability***.

In [1]:
var numbers = new List<int>{1,4,2};

var collectionQuerable = firstCollection.AsQueryable();

var dlsResult = from collect in collectionQuerable
                where !collect.FirstProperty.Equals("First")
                     && numbers.Contains(collect.SecondProperty)
                select collect;

PrintResult(dlsResult.ToList(), result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondProperty}");

#### Agreggate Query 

In [1]:
var numbers = new List<int>{1,4,2};

var dlsResult = from second in secondCollection.AsQueryable()
                join first in firstCollection.AsQueryable()
                    on second.SecondProperty equals first.SecondProperty
                where !first.FirstProperty.Equals("First")
                orderby first.SecondProperty
                select second.Name;

PrintResult(dlsResult.ToList(), result => $"Name: {result}");